<a href="https://colab.research.google.com/github/megmarv/Client-Deposit-Prediction/blob/main/MachineLearningCoursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from google.colab import drive

# Load dataset
data = pd.read_csv('/content/drive/MyDrive/bank-additional-full.csv', delimiter=';')

# Handle missing values
data = data.dropna()

# Encode categorical variables
categorical_columns = ['job', 'marital', 'education', 'contact']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Feature scaling
scaler = StandardScaler()
numerical_columns = ['age', 'balance', 'duration', 'campaign']
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# Split dataset
X = data.drop('y', axis=1)  # Features
y = data['y']               # Target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


KeyError: "['balance'] not in index"

In [7]:
!git clone https://github.com/megmarv/Client-Deposit-Prediction.git

Cloning into 'Client-Deposit-Prediction'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [9]:
%cd Client-Deposit-Prediction


/content/Client-Deposit-Prediction
